# Load and split the dataset

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

# Load the Cats vs. Dogs dataset
dataset, info = tfds.load("cats_vs_dogs", as_supervised=True, with_info=True)

# Split dataset into training and testing
train_data, test_data = dataset['train'].take(20000), dataset['train'].skip(20000)

# Get the labels for the images
label_names = info.features['label'].names
# Define a function to display some sample images
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(train_data.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(label_names[label])
    plt.axis('off')



# Preprocess Images for a Model

In [3]:
IMG_SIZE = (160, 160)  # Resize images to 160x160

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)  # Resize
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    return image, label

# Apply mapping, batching, and prefetching
train_data = train_data.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)



# Build a Simple CNN Model

In [ ]:
from tensorflow.keras import layers, models, optimizers

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
#    layers.Dropout(0.5), # Regularization
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification (Cat or Dog)
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


# Train the model

In [ ]:
model.fit(train_data, epochs=9, validation_data=test_data)


# Save the trained network

In [ ]:
model.save('cats_dogs_cnn_model.h5')

In [ ]:
# load the model
model = tf.keras.models.load_model('cats_dogs_cnn_model.h5')
model.summary()


# Predict on a new image

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load and preprocess the image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(160, 160))  # Resize
    img_array = image.img_to_array(img)  # Convert to NumPy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize to [0,1]
    return img_array

# Path to the new image
img_path = "test_images/1.jpg"

# Preprocess the image
img_array = load_and_preprocess_image(img_path)

# Make prediction
prediction = model.predict(img_array)

# Interpret the result
if prediction[0] > 0.5:
    print("It's a **dog** 🐶", prediction[0])
else:
    print("It's a **cat** 🐱", prediction[0])
